# Three Layer Router - All Qubit Tomography

In [1]:
import sys
sys.path.append('..') # Search for custom module in the top level. 

# Import my custom modules.
from allens_quantum_package.functions import * 
from allens_quantum_package.operators import *

from qiskit import *
from qiskit.quantum_info import state_fidelity
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService

import scipy
import numpy
from numpy import set_printoptions, radians, ndarray, radians, array, sqrt

import itertools

In [2]:
# Set the floating point diplay precision to 3 decimal places, sufficient for our purposes.
set_printoptions(precision=3)

# Initialise the Qiskit runtime service. 
service = QiskitRuntimeService()

In [3]:
states = [
    (radians(-104), radians(-146)),
    (radians(-158), radians(-108)),
    (radians(-110), radians(-172))
]

In [4]:
def build_circuit(theta, phi) -> QuantumCircuit:
    circ = QuantumCircuit(15)

    # Route on first layer
    circ.h(7)
    circ.u(theta, phi, 0, 4)
    circ.cswap(7, 4, 10)

    # Route on second layer, top
    circ.h(3)
    circ.cswap(3, 2, 4)

    # Route on second layer, bottom
    circ.h(11)
    circ.cswap(11, 10, 12)

    # Route on third layer, first
    circ.h(0)
    circ.cswap(0, 1, 2)

    # Route on third layer, second
    circ.h(6)
    circ.cswap(6, 4, 5)

    # Route on third layer, third
    circ.h(8)
    circ.cswap(8, 9, 10)

    # Route on third layer, fourth
    circ.h(14)
    circ.cswap(14, 12, 13)

    return circ

def build_tomography_set(circuit: QuantumCircuit) -> QuantumCircuit:
    signal_paths = (1, 2, 4, 5, 9, 10, 12, 13)

    circuit_x = circuit.copy()
    circuit_x.h(signal_paths)
    circuit_x.measure_all()
    
    circuit_y = circuit.copy()
    circuit_y.sdg(signal_paths)
    circuit_y.h(signal_paths)
    circuit_y.measure_all()
    
    circuit_z = circuit.copy()
    circuit_z.measure_all()
    
    return [circuit_x, circuit_y, circuit_z]


In [5]:
build_circuit(*states[0]).draw()

┌───┐                   
 q_0: ─────────┤ H ├─────────────────■─
               └───┘                 │ 
 q_1: ───────────────────────────────X─
                                     │ 
 q_2: ────────────────────────────X──X─
               ┌───┐              │    
 q_3: ─────────┤ H ├──────────────■────
      ┌────────┴───┴─────────┐    │    
 q_4: ┤ U(-1.8151,-2.5482,0) ├─X──X──X─
      └──────────────────────┘ │     │ 
 q_5: ─────────────────────────┼─────X─
               ┌───┐           │     │ 
 q_6: ─────────┤ H ├───────────┼─────■─
               ├───┤           │       
 q_7: ─────────┤ H ├───────────■───────
               ├───┤           │       
 q_8: ─────────┤ H ├───────────┼─────■─
               └───┘           │     │ 
 q_9: ─────────────────────────┼─────X─
                               │     │ 
q_10: ─────────────────────────X──X──X─
               ┌───┐              │    
q_11: ─────────┤ H ├──────────────■────
               └───┘              │    
q_12: ────────────────────────────X──X─
                                     │ 
q_13: ───────────────────────────────X─
               ┌───┐                 │ 
q_14: ─────────┤ H ├─────────────────■─
               └───┘

In [6]:
circuits_to_send = list(itertools.chain.from_iterable([
        circuit for circuit in [
            build_tomography_set(build_circuit(theta, phi)) for theta, phi in states
        ]
    ]
))

In [7]:
ibm_brisbane = service.get_backend('ibm_brisbane')

In [8]:
circuits_to_send = [transpile(circuit, ibm_brisbane) for circuit in circuits_to_send]

In [9]:
def get_all_physical_qubits(circuit_list: list[QuantumCircuit]) -> set[int]:
    all_indices = set()
    for circ in circuit_list:    
        measurement_indices = [instr[1][0]._index for instr in circ.data if instr[0].name == 'measure']
        all_indices = all_indices.union(measurement_indices)
    return all_indices


def get_mitigation_circuits(circuit_list: list[QuantumCircuit]) -> list[QuantumCircuit]:

    physical_qubits = get_all_physical_qubits(circuit_list)

    num_qubits = len(physical_qubits)
    all_0 = QuantumCircuit(127, num_qubits)
    all_0.measure(physical_qubits, range(num_qubits))

    all_1 = QuantumCircuit(127, num_qubits)
    all_1.x(physical_qubits)
    all_1.measure(physical_qubits, range(num_qubits))

    return [all_0, all_1]

In [10]:
circuits_to_send = get_mitigation_circuits(circuits_to_send) + circuits_to_send

In [11]:
hardware_job = ibm_brisbane.run(circuits=circuits_to_send)

In [12]:
print(f"Hardware job ID: {hardware_job.job_id()}")

Hardware job ID: cw6gda5ggr6g0087cdg0


---

In [13]:
results = hardware_job.result().get_counts()

In [14]:
def get_signal_qubit_idx_from_measurement_outcome(bit_string: str) -> int:
    if bit_string[7] == '0':
        if bit_string[11] == '0':
            if bit_string[8] == '0':
                return 10
            else:
                return 9
        else:
            if bit_string[14] == '0':
                return 12
            else:
                return 13
    else:
        if bit_string[3] == '0':
            if bit_string[6] == '0':
                return 4
            else:
                return 5
        else:
            if bit_string[0] == '0':
                return 2
            else:
                return 1


def get_xyz_counts_for_circuit(counts_list: list) -> tuple[dict]:
    output = []
    for counts in counts_list:
        counts_dict = {}
        
        counts_dict['0'] = sum(count for bit_string, count in counts.items() if bit_string[get_signal_qubit_idx_from_measurement_outcome(bit_string)] == '0')
        counts_dict['1'] = sum(count for bit_string, count in counts.items() if bit_string[get_signal_qubit_idx_from_measurement_outcome(bit_string)] == '1')
        
        output.append(counts_dict)
    
    return tuple(output)


def print_unmitigated_fidelities_combined(counts, theta, phi):

    qubit = density_op_from_counts_dict(*get_xyz_counts_for_circuit(counts))

    print(get_xyz_counts_for_circuit(counts))
    print(qubit)

    psi = gen_qubit(theta, phi)

    fidelity_q = state_fidelity(qubit, psi)

    print(f'State fidelity from combined counts: {fidelity_q}\n')

In [15]:
print_unmitigated_fidelities_combined(results[2:5], *states[0])
print_unmitigated_fidelities_combined(results[5:8], *states[1])
print_unmitigated_fidelities_combined(results[8:11], *states[2])

({'0': 2030, '1': 1970}, {'0': 1613, '1': 2387}, {'0': 2473, '1': 1527})
[[0.618+0.j    0.007+0.097j]
 [0.007-0.097j 0.382+0.j   ]]
State fidelity from combined counts: 0.424930967581284

({'0': 2057, '1': 1943}, {'0': 2100, '1': 1900}, {'0': 2160, '1': 1840})
[[0.54 +0.j    0.014-0.025j]
 [0.014+0.025j 0.46 +0.j   ]]
State fidelity from combined counts: 0.4734690240610905

({'0': 2206, '1': 1794}, {'0': 2098, '1': 1902}, {'0': 1963, '1': 2037})
[[0.491+0.j    0.051-0.025j]
 [0.051+0.025j 0.509+0.j   ]]
State fidelity from combined counts: 0.5542909959923848



In [16]:
# Gets the map from a physical qubit to a classical bit for the mitigation calibration matrices
def get_qubit_to_clbit_mappings_for_mitigation(circuit: QuantumCircuit) -> dict[int, int]:
    return dict(
        ((instr[1][0]._index, instr[2][0]._index) for instr in circuit.data if instr[0].name == 'measure')
    )

# Gets the map from physical qubit to mitigation matrix
def get_assignment_matrices(mappings: dict, counts_0: dict, counts_1: dict) -> dict[int, ndarray]:

    output = {}

    for physical_qubit, classical_bit in mappings.items():
        
        # Determine zero state for assignment matrix
        result_0 = sum([count for bit_string, count in counts_0.items() if bit_string[classical_bit] == '0'])
        result_1 = sum([count for bit_string, count in counts_0.items() if bit_string[classical_bit] == '1'])
        
        # Calculate zero ket
        zero_ket = array([[result_0], 
                          [result_1]]) / (result_0 + result_1)
        
        # Determine one state for assignment matrix
        result_0 = sum([count for bit_string, count in counts_1.items() if bit_string[classical_bit] == '0'])
        result_1 = sum([count for bit_string, count in counts_1.items() if bit_string[classical_bit] == '1'])
        
        # Calculate zero ket
        one_ket = array([[result_0], 
                         [result_1]]) / (result_0 + result_1)
        
        assignment_matrix = numpy.concatenate([zero_ket, one_ket], axis=1)

        output[physical_qubit] = assignment_matrix
    
    return output

# Get the list of physical qubits in the same order as the classical bits 
def get_qubits_in_clbit_order(circuit: QuantumCircuit) -> list[int]:
    
    cl_bit_to_qbit_map = dict((instr[2][0]._index, instr[1][0]._index) for instr in circuit.data if instr[0].name == 'measure')

    return [*cl_bit_to_qbit_map.values()]
    

# Get the tensor of all mitigation matrices in the correct order for each circuit
def get_mitigation_matrix(circuit: QuantumCircuit, assignment_mappings: dict) -> ndarray:
    ordered_physical_qubits = get_qubits_in_clbit_order(circuit)

    ordered_physical_qubits.reverse()

    assignment_matrices = [assignment_mappings[qubit] for qubit in ordered_physical_qubits]

    return tens(*(scipy.linalg.inv(mat) for mat in assignment_matrices))


def get_bit_strings(count: int) -> list[str]:
    return [''.join(bits) for bits in itertools.product(['0', '1'], repeat=count)]


def get_corrected_counts(circuit: QuantumCircuit, counts_for_mitigation: dict, assignment_mappings: dict, num_qubits: int) -> dict[str, int]:
    bit_strings = get_bit_strings(num_qubits)

    counts_list = []
    for bit_string in bit_strings:
        counts_list.append(counts_for_mitigation[bit_string] if bit_string in counts_for_mitigation else 0)
    
    # Get vector of counts
    counts_vector = numpy.concatenate(
        array([[count for count in counts_list]]),
        axis=0
    )

    mitigation_matrix = get_mitigation_matrix(circuit, assignment_mappings)
    
    # Multiply by mitigation matirx
    corrected_vector = mitigation_matrix @ counts_vector
    corrected_vector = corrected_vector.astype(int)
    
    output = {}
    for idx, bit_string in zip(range(2**num_qubits), bit_strings):
        output[bit_string] = int(corrected_vector[idx])
    
    return output

In [17]:
# Get list of qubit to clbit mappings for the mitigation circuits.
mitigation_mappings = get_qubit_to_clbit_mappings_for_mitigation(circuits_to_send[0])

# Calculate assignment matrices for each qubit. 
assignment_matrices = get_assignment_matrices(mitigation_mappings, results[0], results[1])

In [18]:
def print_mitigated_fidelities_combined(circuits, counts, assignment_matrices, num_qubits, theta, phi):

    mitigated_counts = []

    for circuit, count in zip(circuits, counts):
        mitigated_counts.append(get_corrected_counts(circuit, count, assignment_matrices, num_qubits))

    qubit = density_op_from_counts_dict(*get_xyz_counts_for_circuit(mitigated_counts))
    
    print(get_xyz_counts_for_circuit(mitigated_counts))
    print(qubit)

    psi = gen_qubit(theta, phi)

    fidelity_q = state_fidelity(qubit, psi)

    print(f'Mitigated state fidelity from combined counts: {fidelity_q}\n')

In [19]:
print_mitigated_fidelities_combined(circuits_to_send[2:5], results[2:5], assignment_matrices, 15, *states[0])
print_mitigated_fidelities_combined(circuits_to_send[5:8], results[5:8], assignment_matrices, 15, *states[1])
print_mitigated_fidelities_combined(circuits_to_send[8:11], results[8:11], assignment_matrices, 15, *states[2])

({'0': 2029, '1': 1968}, {'0': 1613, '1': 2384}, {'0': 2344, '1': 1482})
[[0.613+0.j    0.008+0.096j]
 [0.008-0.096j 0.387+0.j   ]]
Mitigated state fidelity from combined counts: 0.42655503623135715

({'0': 2056, '1': 1942}, {'0': 2098, '1': 1898}, {'0': 2079, '1': 1781})
[[0.539+0.j    0.014-0.025j]
 [0.014+0.025j 0.461+0.j   ]]
Mitigated state fidelity from combined counts: 0.4747758615631601

({'0': 2201, '1': 1794}, {'0': 2098, '1': 1902}, {'0': 1921, '1': 1994})
[[0.491+0.j    0.051-0.025j]
 [0.051+0.025j 0.509+0.j   ]]
Mitigated state fidelity from combined counts: 0.5537936620393951

